In [1]:
! pip install stackapi
! pip install tensorflow --upgrade
! pip install numpy --upgrade

In [2]:
# import time
# from stackapi import StackAPI
# import socket

# SITE = StackAPI('stackoverflow', key='mticWGYTt*ev0s40SgwkYQ((')

# global client_socket

# while True:
#     now = time.time()
#     time.sleep(10)
#     try:
#         questions = SITE.fetch('questions', fromdate=int(now-10), todate=int(now), filter='withbody')['items']#[0]['tags']
#         combined_questions = [] 
#         keys = ['title', 'body']
#         for i in range(len(questions)):
#             custom_filter = list(map(questions[i].get, keys))
#             combined_questions.append(custom_filter)
#         #print(combined_questions)
#             client_socket.send(msg.encode())
#     except:
#             # print("Error on_data: %s" % str(e))
#             # self.disconnect()
#         pass

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from tensorflow.keras import *
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, plot_confusion_matrix
import random
import numpy as np
import seaborn as sns
import sys
import os
import time
from stackapi import StackAPI
import socket

/opt/conda/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2022-12-11 00:13:18.195921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 00:13:18.367559: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-11 00:13:18.367599: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror

In [4]:
! ls

bin	   dev	   home   lib64       media  proc  sbin  tmp	    valid.csv
boot	   etc	   lib	  libx32      mnt    root  srv	 train.csv  var
copyright  hadoop  lib32  lost+found  opt    run   sys	 usr


In [5]:
train = pd.read_csv('gs://eecs6893-stackoverflow-title-body-bucket/train.csv')
val = pd.read_csv('gs://eecs6893-stackoverflow-title-body-bucket/valid.csv')
data = pd.concat((train, val))
data = data.reset_index()
data['body_scraped']=data['Body'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)
train, test = train_test_split(data, test_size=0.2, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(data['Y'].values) 
train_y = encoder.transform(train['Y'].values)
val_y = encoder.transform(val['Y'].values)
test_y = encoder.transform(test['Y'].values)

train_X = train['body_scraped'].values
val_X = val['body_scraped'].values
test_X = test['body_scraped'].values

train_y = np_utils.to_categorical(train_y)
val_y = np_utils.to_categorical(val_y)
test_y = np_utils.to_categorical(test_y)



/opt/conda/miniconda3/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/opt/conda/miniconda3/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

tokens = Tokenizer(num_words=100000, oov_token='') # i feel like vocab size of 100,000 is probably enough 
tokens.fit_on_texts(list(train_X))

In [7]:
# load in model
model = tf.keras.models.load_model('gs://eecs6893-stackoverflow-title-body-bucket/lstm.h5')


2022-12-11 00:14:00.734530: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-11 00:14:00.734577: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-11 00:14:00.734613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cluster-168b-m): /proc/driver/nvidia/version does not exist
2022-12-11 00:14:00.734960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import time
from stackapi import StackAPI
import socket

SITE = StackAPI('stackoverflow')

def stackexchange_tags():
    global client_socket
    while True:
        now = time.time()
        time.sleep(10)
        try:
            questions = SITE.fetch('questions', fromdate=int(now-10), todate=int(now), filter='withbody')['items']#[0]['tags']

            for i in range(len(questions)):
                msg = ""
                print(questions[i]['body'])
                html_scrap = BeautifulSoup(questions[i]['body'], 'html.parser').text
                html_scrap = [html_scrap]
                str_sequence = tokens.texts_to_sequences(html_scrap)
                str_sequence_pad = preprocessing.sequence.pad_sequences(str_sequence, maxlen=1000, padding='post', truncating='post')

                pred_test = model.predict(str_sequence_pad)

                pred = np.argmax(pred_test, axis=1)
                msg += questions[i]['title'] + '|||||||' + str(np.squeeze(pred)) + '|||||||'
                print(msg)
                    
                client_socket.send(msg.encode())
                    
            return True
        except:
            # print("Error on_data: %s" % str(e))
            # self.disconnect()
            pass

def sendData(c_socket):
    """
    Sends data to socket
    """
    global client_socket
    client_socket = c_socket
    stream = stackexchange_tags()


class stackexchange_client:
    def __init__(self, TCP_IP, TCP_PORT):
        self.s = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.s.bind((TCP_IP, TCP_PORT))

    def run_client(self):
        try:
            self.s.listen(1)
            while True:
                print("Waiting for TCP connection...")
                conn, addr = self.s.accept()
                print("Connected... Starting getting tags.")
                sendData(conn)
                conn.close()
        except KeyboardInterrupt:
            exit


if __name__ == '__main__':
    client = stackexchange_client("localhost", 9001)
    client.run_client()

Waiting for TCP connection...
Connected... Starting getting tags.
<p>I am new to Hive and trying to load an external table via a JSON file having free schema arrays and structures. I had created a table as:</p>
<pre><code>create external table longtimedata_v4
(measurementPointName string,measurementPointId string,dataTypeId string,dataTypeName string,channels ARRAY&lt;            struct&lt;name:string,unit:string,dvalues:ARRAY&lt;struct&lt;atimedt:string,dvalue:string&gt;&gt;&gt;&gt;)
ROW FORMAT SERDE 'org.apache.hive.hcatalog.data.JsonSerDe'
STORED AS TEXTFILE
LOCATION '/warehouse/tablespace/external/hive/longtimedata'
</code></pre>
<p>My JSON file looks like:</p>
<pre><code>{
    &quot;measurementPointName&quot;: &quot;Saab Scania, BBY2908, Sibbhult&quot;,
    &quot;measurementPointId&quot;: 35,
    &quot;dataTypeId&quot;: 1,
    &quot;dataTypeName&quot;: &quot;Voltage&quot;,
    &quot;channels&quot;: [
        {
            &quot;name&quot;: &quot;U1 RMS&quot;,
            &quot;uni

1/1 [==============================] - 3s 3s/step
This is the prediction: 0
Hive data loading using complex json structure|||||||0
Waiting for TCP connection...
Connected... Starting getting tags.
<p>I am writing a test-project on vue 3 and vuex and caught this error:
AxiosError 'Request failed with status code 404', code: 'ERR_BAD_REQUEST'</p>
<p>cart.vue</p>
<pre><code>
&lt;button @click=&quot;sendCartToServer&quot;&gt;&lt;/button&gt;


methods: {

  ...mapActions([
    'SEND_CART_TO_SERVER']),

sendCartToServer(){
  this.SEND_CART_TO_SERVER();

</code></pre>
<p>store.js</p>
<pre><code>state:{
cart:[],
jsnoncart:[]
},
actions:{
SEND_CART_TO_SERVER(state){
   state.jsonCart = JSON.stringify(state.cart);
   return axios.post('https://my-json-server.typicode.com/BIGBASEDFLOPPA/Test-task-vue/',{
     Body:this.state.jsonCart
  })
       .then(response=&gt;{})
       .catch((error)=&gt;{
           console.log(error)
           return error;
       })
}
}
</code></pre>
<p>(state.cart - ar

1/1 [==============================] - 0s 133ms/step
This is the prediction: 1
Unity 2D - Attack system only works while moving|||||||1
Waiting for TCP connection...
Connected... Starting getting tags.
<p>We usually update some settings in WebConfig file in iss in Azure and restart the app service and we find that application is updated.  Seem to me application build again as we do that when working locally in Visual Studio.  However, without much explanation someone told me the application does not rebuild.  So if application does not rebuild then how the changes take affect.</p>
<p>Just to elaborate further, we have keyvault secret. We can change the secret and find the values updated after restarting so what process is occurring when the app service restarted? fro the Azure portal</p>

1/1 [==============================] - 0s 135ms/step
This is the prediction: 0
What happens under the hood when the webapp restart in Azure?|||||||0
Waiting for TCP connection...
Connected... Starting